# The aim of this notebook is to launch the GBOML simulation for a given configuration



In [ ]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

belgian_data_folder = os.path.join(project_root, 'Data', 'Belgian_model')
plot_folder = os.path.join(project_root, 'Plots')
result_folder = os.path.join(project_root, 'Results')
template_folder = os.path.join(project_root, 'Templates')
simulation_folder = os.path.join(project_root, 'Simulations', 'FINAL_DME')
model_folder = os.path.join(project_root, 'Models GBOML')

sys.path.append(os.path.join(project_root, 'Modules'))

import GBOML_function as gf
import process_funct as pf

current_dir = os.getcwd()
result_dir = os.path.join(current_dir, 'Results')

## Set up

### Class and functions

In [ ]:

import argparse
import sys
import json
import time
from termcolor import colored
import logging
import sys
import pandas as pd
from tqdm import tqdm
from copy import deepcopy 
from gboml import GbomlGraph
from gboml.output import write_csv
from gboml.compiler import parse_file
from gboml.compiler.classes import Parameter, Expression, Constraint, Identifier, Node

class MyGbomlGraph(GbomlGraph):
    def update_global_parameter(self, param: Parameter):
        self.global_parameters = [x if x.name != param.name else param for x in self.global_parameters]
    def get_node_by_name(self, name: str):
        for node in self.list_nodes:
            if node.name == name:
                return node
        return None
    def get_hyperedge_by_name(self, name: str):
        for h in self.list_hyperedges:
            if h.name == name:
                return h
        return None
    
    def export_model_to_lp(self, filename: str):
        """
        Optional method: exports model to LP format (requires Gurobi model to be built first).
        """
        if hasattr(self, 'model'):  # Gurobi model exists after build_model()
            self.model.write(filename)
            print(f"✅ Model written to {filename}")
        else:
            print("⚠️ Gurobi model not found. Make sure build_model() was called.")

def get_named_symbol_in_list(l, name):
    for h in l:
        if h.name == name:
            return h
    return None

def get_node_in(element, name: str):
    if isinstance(element, GbomlGraph):
        l = element.list_nodes
    elif isinstance(element, Node):
        l = element.nodes
    else:
        raise Exception(f"Unknown class {element.__class__}")
    return get_named_symbol_in_list(l, name)
    

def get_hyperedge_in(element, name: str):
    if isinstance(element, GbomlGraph):
        l = element.list_hyperedges
    elif isinstance(element, Node):
        l = element.hyperedges
    else:
        raise Exception(f"Unknown class {element.__class__}")
    return get_named_symbol_in_list(l, name)


def update_parameter(symb, param):
    symb.parameters = [x if x.name != param.name else param for x in symb.parameters] 


def update_constraint(element, constraint_name, new_operator):
    found = False
    for i, c in enumerate(element.constraints):
            if c.name == constraint_name: 
                c.operator = new_operator
                found = True 
                print("✅ Constraint updated.")
                break
    if not found:
        print(f"❌ No '{constraint_name}' constraint found.")



def replace_constraint_operator(graph: MyGbomlGraph, constraint_name: str, new_operator: str = "==", element = None):
    """
    Modifie l'opérateur d'une contrainte (ex : <= devient ==) dans un graphe GBOML.

    Args:
        graph: objet MyGbomlGraph
        constraint_name: nom exact de la contrainte à modifier
        new_operator: opérateur de remplacement (par défaut '==')
    """
    # Cherche la contrainte dans tous les hyperedges 
    
    if element is not None:
        update_constraint(element, constraint_name, new_operator)
        return
    else: 
        for h in graph.list_hyperedges:
            update_constraint(h, constraint_name, new_operator)

### Set name period


In [ ]:
def set_periodicity(TimeHorizon):
    
    if TimeHorizon >= 8760:
        if TimeHorizon % 8760 == 0:
            time = f"{int(TimeHorizon/8760)}years"

        elif TimeHorizon % 8760 != 0:
            time = f"{TimeHorizon}hours"
                
    if TimeHorizon < 8760:
            if TimeHorizon % 24 == 0:
                time = f"{int(TimeHorizon/24)}days"
            
            else:
                time = f"{TimeHorizon}hours"
                
    return time

### Build model

In [ ]:
def build_model(TimeHorizon,model_name):
    program = parse_file(model_name)
    model = MyGbomlGraph()
    model.add_nodes_in_model(*deepcopy(program.get_nodes()))
    model.add_hyperedges_in_model(*deepcopy(program.get_links()))
    model.add_global_parameters(deepcopy(program.get_global_parameters()))
    model.set_timehorizon(TimeHorizon)

    return model


def build_and_solve(model, outfile=None):
    """
    Builds and solves the given GBOML model using Gurobi.
    If the solution is optimal, it exports a JSON and CSV with results.
    If infeasible, it prints the issue and exports a .lp file for debugging.

    :param model: GBOML model (MyGbomlGraph)
    :param outfile: Optional output filename (JSON). CSV will use same name.
    :return: Dictionary of results (or empty if infeasible)
    """
    print("🚧 Building model...")
    model.build_model(8)

    # Export LP version of the model for manual debug if needed
    model.export_model_to_lp("debug_model.lp")
    print("📤 Model exported to 'debug_model.lp'")

    print("🚀 Solving...")
    solution, objective, status, solver_info, cai, vai = model.solve_gurobi(
        details=os.path.join(project_root, "gurobi_detail.txt"),
        opt_file=os.path.join(project_root, "gurobi.opt")
    )

    print("🧠 Gurobi status:", status)
    if "OPTIMAL" not in str(status).upper():
        print("❌ No feasible solution found.")
        return {}

    print("✅ Optimal solution found.")

    out = model.turn_solution_to_dictionary(solver_info, status, solution, objective, cai, vai)
    outlist = model.turn_solution_to_list(solution, constraints_info=cai)
    list_names = [x[0] for x in outlist]
    list_val = [x[1] for x in outlist]

    if outfile:
        with open(outfile, 'w') as f_json:
            json.dump(out, f_json, indent=4, sort_keys=True)
        write_csv(outfile.replace(".json", ".csv"), list_names, list_val, transpose=True)
        print(f"📁 Results saved: {outfile} and CSV version.")      

    return out

### Run scenarios function

In [ ]:
def run_scenarios(base_model, file_path, period, new_scenario=None, modifications=None, scenario_name=None):
    """
    Runs a GBOML simulation scenario. Optionally modifies certain node parameters.

    :param base_model: Compiled GBOML model (MyGbomlGraph)
    :param file_path: Path to save output files (with trailing slash)
    :param period: Suffix for the output file (e.g. _2030)
    :param new_scenario: Optional name of the scenario (used as suffix in filenames)
    :param modifications: Optional list of parameter changes (only if new_scenario is not None). Format:
                          [{"cluster": "INLAND", "node": "DME_PLANTS", "params": {"max_capacity": 0}}]
    """
    from copy import deepcopy
    import time

    if new_scenario:
        scenario = new_scenario
    elif scenario_name:
        scenario = scenario_name
    else:
        scenario = "base" 
    
    print(f"\n🔁 Running scenario: " + colored(f'{scenario}', 'white', 'on_blue'))
    start_time = time.time()

    model = deepcopy(base_model)

    if new_scenario and modifications:
        for item in modifications:
            cluster_name = item["cluster"]
            node_name = item["node"]
            for param_name, value in item["params"].items():
                try:
                    update_subnode_parameter(model, cluster_name, [node_name], param_name, value)
                except Exception as e:
                    print(colored(f"⚠️ Skipped modification: {cluster_name}.{node_name}.{param_name} – {e}", 'yellow'))

    out_path = os.path.join(file_path, f"scenario_{scenario}_{period}.json")
    results = build_and_solve(model, outfile=out_path)

    duration = time.time() - start_time
    print(f"✅ Scenario '{scenario}' completed in {duration:.1f} seconds.")
    print(f"'scenario_{scenario}_{period}.json'")
    
    return results

# MAIN

In [ ]:
TimeHorizon = 8760
period = set_periodicity(TimeHorizon)

simulation_folder = os.path.join(simulation_folder)  # Path to simulation folder where the .json and .csv files will be saved
model_folder = os.path.join(model_folder)  # Path to the folder where the GBOML model file is located

# Hypotheses that may change between scenarios:


- Grid max capacity: 23 GW 
- 75 €/MWh import cost for green hydrogen from NL
- 0 €/tonne CO2 tax 
- PCCC capture rate: (changed from 90% to 95%)
- CO2 export cost: (changed from 2 to 8.6 €/tonne)
- Change Loenhout capacity: (8000 to 7600 GWh/y and 7 to 7.25 GW) 
- Jet fuel demand with biomass cost (0.876 M€/kt(biomass)) 
- Biomass cost for BIOMETHANE plant: (0.0237 M€/kt(biomass) and 46.4% conversion efficiency) 
- Linepack flexibility (minimum level)


# How to launch a scenario - step by step


<span style="color:#0b3c5d;"><b>Upload the file</b></span>
```python
file = "FILE_NAME"
model = build_model(TimeHorizon, os.path.join(model_folder, file))
``` 

<span style="color:#0b3c5d;"><b>Update global parameters</b></span>
```python
co2_emission_cost = 0  # €/tonne
model.update_global_parameter(Parameter("co2_emission_cost",Expression("literal", co2_emission_cost)))
```
 
<span style="color:#0b3c5d;"><b>Update specific parameters</b></span>
```python
max_pv_capacity = 50
Inland = get_node_in(model, "INLAND")
PV = get_node_in(Inland, "PV")
model.update_parameter(PV,Parameter("max_capacity",Expression("literal", "max_pv_capacity")))
```
 
<span style="color:#0b3c5d;"><b>Change constraint operator sign</b></span>
```python
replace_constraint_operator(graph=model,constraint_name="electricity",new_operator=">=")
```
 
<span style="color:#0b3c5d;"><b>Change constraint operator sign in a specific node / hyperedge</b></span>
```python
Inland = get_node_in(model, "INLAND")
INLAND_BALANCE = get_hyperedge_in(Inland, "BALANCES")
replace_constraint_operator(graph=model,constraint_name="dme_balanced",new_operator=">=",element=INLAND_BALANCE)
```

# DME Experiments and simulation launch

### DME base case
- Fix no co2 taxes
- Change co2 export price
- Allow offshore hydrogen production 
- No economies of scale on direct pathways   

In [ ]:
file = "scenario_dme.txt"

model = build_model(TimeHorizon, os.path.join(model_folder, file))


# No CO2 taxes
co2_emission_cost = 0 # €/tonne
model.update_global_parameter(Parameter("co2_emission_cost", Expression("literal", co2_emission_cost)))

# CO2 export price
CO2_export = get_node_in(Inland,"CO2_EXPORT")
update_parameter(CO2_export, Parameter("export_cost", Expression("literal", 8.6 * 1/(10**3))))

# Offshore hydrogen 
model.update_global_parameter(Parameter("case_elec", Expression("literal", 0))) 

# Grid capacity
model.update_global_parameter(Parameter("grid_limit", Expression("literal", 23))) 

# DME Plants capex adjustment
Inland = get_node_in(model, "INLAND")
capex = 60969 / 8.01 # (M€/kton)/ (LHV in GWh/kton)
DME_PLANTS = get_node_in(Inland,"DME_PLANTS")
update_parameter(DME_PLANTS, Parameter("capex", Expression("literal", capex)))

scenario = run_scenarios(model, file_path=simulation_folder, period=period, scenario_name=f'dme_no_economies_of_scale')

### DME with no economies of scale on direct pathways and no DMEOH Plants

In [ ]:
file = "scenario_dme.txt"

model = build_model(TimeHorizon, os.path.join(model_folder, file))

# No CO2 taxes
co2_emission_cost = 0 # €/tonne
model.update_global_parameter(Parameter("co2_emission_cost", Expression("literal", co2_emission_cost))) 

# CO2 export price
CO2_export = get_node_in(Inland,"CO2_EXPORT")
update_parameter(CO2_export, Parameter("export_cost", Expression("literal", 8.6 * 1/(10**3))))

# Offshore hydrogen 
model.update_global_parameter(Parameter("case_elec", Expression("literal", 0))) 

# Grid capacity
model.update_global_parameter(Parameter("grid_limit", Expression("literal", 23))) 

# Methanol imposed to zero
model.update_global_parameter(Parameter("meoh", Expression("literal", 0)))

# DME Plants capex adjustment
Inland = get_node_in(model, "INLAND")
capex = 60969 / 8.01 # (M€/kton)/ (LHV in GWh/kton)
DME_PLANTS = get_node_in(Inland,"DME_PLANTS")
update_parameter(DME_PLANTS, Parameter("capex", Expression("literal", capex)))

scenario = run_scenarios(model, file_path=simulation_folder, period=period, scenario_name=f'dme_no_economies_of_scale_no_methanol')

### DME with economies of scale and flexible operation of direct pathways        

In [ ]:
file = "scenario_dme.txt"

model = build_model(TimeHorizon, os.path.join(model_folder, file))

Inland = get_node_in(model, "INLAND")

# No CO2 taxes
co2_emission_cost = 0 # €/tonne
model.update_global_parameter(Parameter("co2_emission_cost", Expression("literal", co2_emission_cost))) 

# CO2 export price
CO2_export = get_node_in(Inland,"CO2_EXPORT")
update_parameter(CO2_export, Parameter("export_cost", Expression("literal", 8.6 * 1/(10**3))))

# Offshore hydrogen 
model.update_global_parameter(Parameter("case_elec", Expression("literal", 0))) 

# Grid capacity
model.update_global_parameter(Parameter("grid_limit", Expression("literal", 23))) 

# Methanol imposed to zero
model.update_global_parameter(Parameter("meoh", Expression("literal", 0)))


# DME flexibility
DME = get_node_in(Inland,"DME_PLANTS")
update_parameter(DME, Parameter("minimum_level", Expression("literal", 0.1)))
update_parameter(DME, Parameter("ramp_rate_up", Expression("literal", 1.0)))
update_parameter(DME, Parameter("ramp_rate_down", Expression("literal", 1.0)))

# Linepack
LINEPACK = get_node_in(Inland, "LINEPACK_NG")
lhv_ng_m3 = 10.166 
min_level = 3.5 * lhv_ng_m3 - 16.3 
update_parameter(LINEPACK, Parameter("minimal_level", Expression("literal", min_level)))  

scenario = run_scenarios(model, file_path=simulation_folder, period=period, scenario_name=f'dme_flex')

### DME with grid capacity increased to 50 GW
- Fix no co2 taxes
- Change co2 export price
- Allow offshore hydrogen production
- H2 import cost from NL: 75 €/MWh
- Grid max capacity: 50 GW

In [ ]:
file = "scenario_dme.txt"

model = build_model(TimeHorizon, os.path.join(model_folder, file))

Offshore = get_node_in(model, "OFFSHORE")
Inland = get_node_in(model, "INLAND")

# No CO2 taxes
co2_emission_cost = 0 # €/tonne
model.update_global_parameter(Parameter("co2_emission_cost", Expression("literal", co2_emission_cost))) 
CO2_export = get_node_in(Inland,"CO2_EXPORT")
update_parameter(CO2_export, Parameter("export_cost", Expression("literal", 8.6 * 1/(10**3))))

# Offshore hydrogen 
model.update_global_parameter(Parameter("case_elec", Expression("literal", 0))) 

# Grid capacity
model.update_global_parameter(Parameter("grid_limit", Expression("literal", 50))) 

 

# Change h2 import cost from France 
H2_NL = get_node_in(Inland, "H2_INTERCONNECTION_NL")
h2_import_costs = [0.075]  
for cost in h2_import_costs:
    update_parameter(H2_NL, Parameter("import_cost", Expression("literal", cost))) 
    h2_cost = cost * 1000
    scenario = run_scenarios(model, file_path=simulation_folder, period=period, scenario_name=f'dme_grid_limit_50GW')    

### DME with Linepack minimal level modified 
- Fix no co2 taxes
- Change co2 export price
- Allow offshore hydrogen production
- H2 import cost from NL: 75 €/MWh
- Linepack minimal level flexibility -> The minimum level is changed from 0 TWh to (Max capacity - 16.3) TWh

In [ ]:
file = "scenario_dme.txt"

model = build_model(TimeHorizon, os.path.join(model_folder, file))

Offshore = get_node_in(model, "OFFSHORE")
Inland = get_node_in(model, "INLAND")

# No CO2 taxes
co2_emission_cost = 0 # €/tonne
model.update_global_parameter(Parameter("co2_emission_cost", Expression("literal", co2_emission_cost))) 
CO2_export = get_node_in(Inland,"CO2_EXPORT")
update_parameter(CO2_export, Parameter("export_cost", Expression("literal", 8.6 * 1/(10**3))))

# Offshore hydrogen 
model.update_global_parameter(Parameter("case_elec", Expression("literal", 0))) 

# Change h2 import cost from France  
# Linepack
LINEPACK = get_node_in(Inland, "LINEPACK_NG")
lhv_ng_m3 = 10.166 
flex = [16.3]
for lvl in flex:
    min_level = 3.5 * lhv_ng_m3 - lvl 
    update_parameter(LINEPACK, Parameter("minimal_level", Expression("literal", min_level))) 
    scenario = run_scenarios(model, file_path=simulation_folder, period=period, scenario_name=f'dme_h2_linepack_{lvl}_GWh_flex')    

# Methanol Experiments and simulation launch

- 75 €/MWh import cost for green hydrogen from NL

In [ ]:
file = "scenario_methanol.txt"

model = build_model(TimeHorizon, os.path.join(model_folder, file))

Offshore = get_node_in(model, "OFFSHORE")
Inland = get_node_in(model, "INLAND")

# No CO2 taxes
co2_emission_cost = 0 # €/tonne
model.update_global_parameter(Parameter("co2_emission_cost", Expression("literal", co2_emission_cost))) 

# CO2 export price
CO2_export = get_node_in(Inland,"CO2_EXPORT")
update_parameter(CO2_export, Parameter("export_cost", Expression("literal", 8.6 * 1/(10**3))))

# Offshore hydrogen 
model.update_global_parameter(Parameter("case_elec", Expression("literal", 0))) 

# Linepack
LINEPACK = get_node_in(Inland, "LINEPACK_NG")
lhv_ng_m3 = 10.166 
min_level = 3.5 * lhv_ng_m3 - 16.3 
update_parameter(LINEPACK, Parameter("minimal_level", Expression("literal", min_level))) 

# Change h2 import cost from France 
H2_NL = get_node_in(Inland, "H2_INTERCONNECTION_NL")
h2_import_costs = 0.75   
update_parameter(H2_NL, Parameter("import_cost", Expression("literal", h2_import_costs))) 
h2_cost = h2_import_costs * 1000
scenario = run_scenarios(model, file_path=simulation_folder, period=period, scenario_name=f'methanol_base_case')    

- 135 €/MWh import cost for green hydrogen from NL -> EnergyVille

In [ ]:
file = "scenario_methanol.txt"

model = build_model(TimeHorizon, os.path.join(model_folder, file))

Offshore = get_node_in(model, "OFFSHORE")
Inland = get_node_in(model, "INLAND")

# No CO2 taxes
co2_emission_cost = 0 # €/tonne
model.update_global_parameter(Parameter("co2_emission_cost", Expression("literal", co2_emission_cost))) 

# CO2 export price
CO2_export = get_node_in(Inland,"CO2_EXPORT")
update_parameter(CO2_export, Parameter("export_cost", Expression("literal", 8.6 * 1/(10**3))))

# Offshore hydrogen 
model.update_global_parameter(Parameter("case_elec", Expression("literal", 0))) 

# Linepack
LINEPACK = get_node_in(Inland, "LINEPACK_NG")
lhv_ng_m3 = 10.166 
min_level = 3.5 * lhv_ng_m3 - 16.3 
update_parameter(LINEPACK, Parameter("minimal_level", Expression("literal", min_level))) 

# Change h2 import cost from France 
H2_NL = get_node_in(Inland, "H2_INTERCONNECTION_NL")
h2_import_costs = 0.135   
update_parameter(H2_NL, Parameter("import_cost", Expression("literal", h2_import_costs))) 
h2_cost = h2_import_costs * 1000
scenario = run_scenarios(model, file_path=simulation_folder, period=period, scenario_name=f'methanol_h2_import_cost_{h2_cost}')    

# Ammonia Experiments and simulation launch

- 75 €/MWh import cost for green hydrogen from NL

In [ ]:
file = "scenario_ammonia.txt"

model = build_model(TimeHorizon, os.path.join(model_folder, file))

Offshore = get_node_in(model, "OFFSHORE")
Inland = get_node_in(model, "INLAND")

# No CO2 taxes
co2_emission_cost = 0 # €/tonne
model.update_global_parameter(Parameter("co2_emission_cost", Expression("literal", co2_emission_cost))) 

# CO2 export price
CO2_export = get_node_in(Inland,"CO2_EXPORT")
update_parameter(CO2_export, Parameter("export_cost", Expression("literal", 8.6 * 1/(10**3))))

# Offshore hydrogen 
model.update_global_parameter(Parameter("case_elec", Expression("literal", 0))) 


# Change h2 import cost from France  
# Linepack
LINEPACK = get_node_in(Inland, "LINEPACK_NG")
lhv_ng_m3 = 10.166 
min_level = 3.5 * lhv_ng_m3 - 16.3 
update_parameter(LINEPACK, Parameter("minimal_level", Expression("literal", min_level))) 

# Change h2 import cost from France 
H2_NL = get_node_in(Inland, "H2_INTERCONNECTION_NL")
h2_import_costs = 0.135   
update_parameter(H2_NL, Parameter("import_cost", Expression("literal", h2_import_costs))) 
h2_cost = h2_import_costs * 1000
scenario = run_scenarios(model, file_path=simulation_folder, period=period, scenario_name=f'ammonia')   

In [ ]:
file = "scenario_ammonia.txt"

model = build_model(TimeHorizon, os.path.join(model_folder, file))

Offshore = get_node_in(model, "OFFSHORE")
Inland = get_node_in(model, "INLAND")

# No CO2 taxes
co2_emission_cost = 0 # €/tonne
model.update_global_parameter(Parameter("co2_emission_cost", Expression("literal", co2_emission_cost))) 

# CO2 export price
CO2_export = get_node_in(Inland,"CO2_EXPORT")
update_parameter(CO2_export, Parameter("export_cost", Expression("literal", 8.6 * 1/(10**3))))

# Offshore hydrogen 
model.update_global_parameter(Parameter("case_elec", Expression("literal", 0))) 

# ASU flexibility
ASU = get_node_in(Inland,"N2_PLANTS")
update_parameter(ASU, Parameter("minimum_level", Expression("literal", 0.1)))
update_parameter(ASU, Parameter("ramp_rate_up", Expression("literal", 1.0)))
update_parameter(ASU, Parameter("ramp_rate_down", Expression("literal", 1.0)))

# Change h2 import cost from France  
# Linepack
LINEPACK = get_node_in(Inland, "LINEPACK_NG")
lhv_ng_m3 = 10.166 
min_level = 3.5 * lhv_ng_m3 - 16.3 
update_parameter(LINEPACK, Parameter("minimal_level", Expression("literal", min_level))) 

# Change h2 import cost from France 
H2_NL = get_node_in(Inland, "H2_INTERCONNECTION_NL")
h2_import_costs = 0.135   
update_parameter(H2_NL, Parameter("import_cost", Expression("literal", h2_import_costs))) 
h2_cost = h2_import_costs * 1000
scenario = run_scenarios(model, file_path=simulation_folder, period=period, scenario_name=f'ammonia_flex')   

- 135 €/MWh import cost for green hydrogen from NL

In [ ]:
file = "scenario_ammonia.txt"

model = build_model(TimeHorizon, os.path.join(model_folder, file))

Offshore = get_node_in(model, "OFFSHORE")
Inland = get_node_in(model, "INLAND")

# No CO2 taxes
co2_emission_cost = 0 # €/tonne
model.update_global_parameter(Parameter("co2_emission_cost", Expression("literal", co2_emission_cost))) 

# CO2 export price
CO2_export = get_node_in(Inland,"CO2_EXPORT")
update_parameter(CO2_export, Parameter("export_cost", Expression("literal", 8.6 * 1/(10**3))))

# Offshore hydrogen 
model.update_global_parameter(Parameter("case_elec", Expression("literal", 0))) 

# Change h2 import cost from France  
# Linepack
LINEPACK = get_node_in(Inland, "LINEPACK_NG")
lhv_ng_m3 = 10.166 
min_level = 3.5 * lhv_ng_m3 - 16.3 
update_parameter(LINEPACK, Parameter("minimal_level", Expression("literal", min_level))) 

# Change h2 import cost from France 
H2_NL = get_node_in(Inland, "H2_INTERCONNECTION_NL")
h2_import_costs = 0.135   
update_parameter(H2_NL, Parameter("import_cost", Expression("literal", h2_import_costs))) 
h2_cost = h2_import_costs * 1000
scenario = run_scenarios(model, file_path=simulation_folder, period=period, scenario_name=f'ammonia_h2_import_cost_{h2_cost}')   